# OutputParser

In [38]:
# base class, 이것을 extends 해 custom parser를 만든다.
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    # parse라는 method를 반드시 구현해야 한다.
    def parse(self, text):
        items = text.strip().split(",") # strip() -> 텍스트 좌우 공백 제거
        return list(map(str.strip, items))
        # items: split(",")으로 나눈 문자열 리스트 --> ["Hello", " how", " are", " you"]
        # map(str.strip, items): items의 각 요소에 strip 을 적용해 공백 제거 --> ["Hello", "how", "are", "you"]
        # list(...): map 객체를 리스트로 변환
    
        # ========== 아래와 같이 작성하면 tuple 형태로 반환 ==========
        # return tuple(map(str.strip, items))

p = CommaOutputParser()
p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

# LCEL(LangChain Expression Language)

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}")
])

### LCEL 적용 전

In [40]:
prompt = template.format_messages(
    max_items = 10,
    question = "What are the Colors?"
)
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

### LCEL 적용 후

In [41]:
# langchain이 내부적으로 format_messages, predict, parse 등을 알아서 호출한다.
chain = template | chat | CommaOutputParser()
chain.invoke({ # invoke의 argument로는 dictionary type이 들어가야 한다.
    "max_items" : 10,
    "question"  : "What are the games?"
})
# chain과 chain을 결합할 수도 있다. by '|'

['fortnite',
 'minecraft',
 'among us',
 'call of duty',
 'league of legends',
 'valorant',
 'overwatch',
 'apex legends',
 'animal crossing',
 'super mario']